# Explore here

In [114]:
import pandas as pd
from sqlalchemy import create_engine, text
import os
import psycopg2
import json

In [80]:
url_movies = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'
url_credits = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'

df_movies = pd.read_csv(url_movies)
df_credits = pd.read_csv(url_credits)

df_movies.to_csv('../data/raw/movies.csv')
df_credits.to_csv('../data/raw/credits.csv')

In [115]:
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
df_movies.head()

In [13]:
# Configurar la conexión a la base de datos
database_username = 'postgres'
database_password = 'Dinamarca1'
database_ip = 'localhost'
database_name = 'my_db'
database_port = '5432'

# Crear la cadena de conexión
connection = psycopg2.connect(
    host=database_ip,
    port=database_port,
    database=database_name,
    user=database_username,
    password=database_password
)

# Definir las columnas que deseas seleccionar
columns = [
    'credits.movie_id',
    'credits.cast',
    'credits.crew',
    'movies.overview', 
    'movies.genres',
    'movies.keywords', 
    'credits.title']

# Definir la consulta SQL para unir las dos tablas y crear la nueva tabla 'combined_table'
query = f'''
CREATE TABLE new_table AS
SELECT {', '.join(columns)}
FROM credits
INNER JOIN movies ON credits.title = movies.title;
'''

# Ejecutar la consulta SQL
with connection:
    with connection.cursor() as cursor:
        cursor.execute(query)

print("La nueva tabla 'combined_table' ha sido creada exitosamente.")

La nueva tabla 'combined_table' ha sido creada exitosamente.


In [157]:
# Definir la consulta SQL para seleccionar todos los datos de la tabla 'combined_table_filtered'
query = "SELECT * FROM new_table"

# Cargar los datos en un DataFrame de Pandas
df_combined = pd.read_sql_query(query, connection)

C:\Users\crroj\AppData\Local\Temp\ipykernel_17952\3849865676.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_combined = pd.read_sql_query(query, connection)


In [158]:
df_combined.head()


,movie_id,cast,crew,overview,genres,keywords,title
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter


In [159]:
df_combined.to_csv('../data/interim/data_films.csv')

In [160]:
# Función para extraer solo los valores 'name' de una columna JSON
def extract_names(json_str):
    items = json.loads(json_str)
    return [item['name'] for item in items]

# Función para extraer los primeros 3 nombres de una columna JSON
def extract_first_three_names(json_str):
    items = json.loads(json_str)
    return [item['name'] for item in items[:3]]

# Aplicar la función a las columnas 'genres' y 'keywords'
df_combined['genres'] = df_combined['genres'].apply(extract_names)
df_combined['keywords'] = df_combined['keywords'].apply(extract_names)
# Aplicar la función a la columna 'cast' para obtener solo los primeros 3 nombres
df_combined['cast'] = df_combined['cast'].apply(extract_first_three_names)

In [161]:
df_combined.head()

,movie_id,cast,crew,overview,genres,keywords,title
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar
1,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End
2,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Spectre
3,49026,"[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises
4,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",John Carter


In [162]:
# Convertir la columna 'crew' en listas de diccionarios
df_combined["crew"] = df_combined["crew"].apply(json.loads)

# Extraer el nombre del director de cada lista de diccionarios
df_combined["director"] = df_combined["crew"].apply(lambda x: " ".join([crew_member['name'] for crew_member in x if crew_member['job'] == 'Director']))

# Convertir la columna 'overview' en una lista de palabras
df_combined['overview'] = df_combined['overview'].apply(lambda x: x if x is not None else [])

In [163]:
df_combined.drop(columns=['crew'], inplace=True)
df_combined.head()

,movie_id,cast,overview,genres,keywords,title,director
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,James Cameron
1,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]","Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,Gore Verbinski
2,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Spectre,Sam Mendes
3,49026,"[Christian Bale, Michael Caine, Gary Oldman]",Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,Christopher Nolan
4,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",John Carter,Andrew Stanton


In [164]:
df_combined['genres'] = df_combined['genres'].apply(lambda x: [genre.replace(" ", "") for genre in x])
df_combined['keywords'] = df_combined['keywords'].apply(lambda x: [keyword.replace(" ", "") for keyword in x])
df_combined['cast'] = df_combined['cast'].apply(lambda x: [actor.replace(" ", "") for actor in x])
df_combined['director'] = df_combined['director'].apply(lambda x: [crew_member.replace(" ", "") for crew_member in x])
df_combined['director'] = df_combined['director'].apply(lambda x: ''.join(x))

In [165]:
df_combined.head()

,movie_id,cast,overview,genres,keywords,title,director
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]","In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,JamesCameron
1,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]","Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,GoreVerbinski
2,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,SamMendes
3,49026,"[ChristianBale, MichaelCaine, GaryOldman]",Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,ChristopherNolan
4,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]","John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",John Carter,AndrewStanton


3. Convertir listas a cadenas

In [166]:
# Convertir cada lista en una cadena de texto
df_combined['overview'] = df_combined['overview'].apply(lambda x: ''.join(x))
df_combined['genres'] = df_combined['genres'].apply(lambda x: ' '.join(x))
df_combined['keywords'] = df_combined['keywords'].apply(lambda x: ' '.join(x))
df_combined['cast'] = df_combined['cast'].apply(lambda x: ' '.join(x))
df_combined['director'] = df_combined['director'].apply(lambda x: ''.join(x))

# Crear la columna 'tag' combinando las columnas convertidas
df_combined['tag'] = df_combined['overview'] + ' ' + df_combined['genres'] + ' ' + df_combined['keywords'] + ' ' + df_combined['cast'] + ' ' + df_combined['director']

# Eliminar las columnas 'overview', 'genres', 'keywords', 'cast' y 'director'
df_combined.drop(columns=['overview', 'genres', 'keywords', 'cast', 'director'], inplace=True)

In [167]:
df_combined.head()

,movie_id,title,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [168]:
df_combined['tag'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [169]:
df_combined.to_csv('../data/processed/data_films.csv')